In [1]:
import os

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

from core.live_data_processor import LiveDataLoader
from core.model import Model

Using TensorFlow backend.


In [2]:
if __name__ == '__main__':
    if not os.path.exists('model'): os.makedirs('model')
    #  Setting pyplot fig size
    plt.rcParams['figure.figsize'] = [10, 8]    
    # Download gold price from yahoo finance
    # gld_dataframe = yf.download(tickers="GLD", start="2000-1-1")
    # Display sample dataframe
    # display(gld_dataframe.head(5))
    # display(gld_dataframe.tail(5))

In [3]:
    data = LiveDataLoader(
        filename=os.path.join('data', 'gld.csv'),
        split=0.8,
        cols=['Open', 'Close', 'Volume']
    )
    x, y = data.get_train_data(30, True)
    display_x_train = np.reshape(x, (-1, 3))
    display(pd.DataFrame(display_x_train).head(5))
    display_y_train = np.reshape(y, (-1, 1))
    display(pd.DataFrame(display_y_train).head(5))

,0,1,2
0,0.000000,0.000000,0.000000
1,0.001350,0.009013,0.945144
2,0.007202,0.012844,1.002003
3,0.010128,0.008337,-0.471095
4,0.011254,0.015097,0.018875


,0
0,-0.013054
1,-0.033940
2,-0.043575
3,-0.047460
4,-0.054529


In [4]:
    model = Model()
    model.build_model()
    model.model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[Model] Model Compiled
Time taken: 0:00:02.149951
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 29, 100)           41600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 29, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 122,101
Trainable param

In [5]:
    '''
	# in-memory training
	model.train(
		x,
		y,
		epochs = configs['training']['epochs'],
		batch_size = configs['training']['batch_size'],
		save_dir = configs['model']['save_dir']
	)
	'''
    # out-of memory generative training
    steps_per_epoch = math.ceil(((data.len_train - 30) / 100))
    model.train_generator(
        data_gen=data.generate_train_batch(
            seq_len=30,
            batch_size=100,
            normalise=True
        ),
        epochs=5,
        batch_size=32,
        steps_per_epoch=steps_per_epoch,
        save_file_name='model.h5'
    )    

[Model] Training Started
[Model] 5 epochs, 32 batch size, 29 batches per epoch
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
29/29 [==============================] - 9s 309ms/step - loss: 0.0337
Epoch 2/5
29/29 [==============================] - 5s 159ms/step - loss: 0.0204
Epoch 3/5
29/29 [==============================] - 6s 200ms/step - loss: 0.0177
Epoch 4/5
29/29 [==============================] - 5s 161ms/step - loss: 0.0169
Epoch 5/5
29/29 [==============================] - 5s 160ms/step - loss: 0.0162
[Model] Training Completed. Model saved as model\model.h5
Time taken: 0:00:32.506523
